In [14]:
import numpy as np
from numpy.testing import assert_allclose
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, Dense
from keras.callbacks import ModelCheckpoint
!pip install plotly --upgrade  # make sure your plotly is up to date

import plotly.express as px
import numpy as np
import pandas as pd
from scipy.io import loadmat
import matplotlib.pyplot as plt


#!pip install plotly --upgrade  # make sure your plotly is up to date

import plotly.express as px
import numpy as np
import pandas as pd
from scipy.io import loadmat
import matplotlib.pyplot as plt

Requirement already up-to-date: plotly in /usr/local/lib/python3.7/dist-packages (4.14.3)


In [5]:
import os
os.listdir()

['.config', 'model_complete.h5', 'sample_data']

In [6]:
new_model = load_model('model_complete.h5')

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
from google.colab import drive
drive.mount('/content/drive')

spectra = loadmat('/content/drive/MyDrive/Spectra_3s_25c.mat')
#more_spectra = loadmat('/content/drive/MyDrive/5_more_spectra.mat')
spectra.keys()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


dict_keys(['__header__', '__version__', '__globals__', 'AmpSAmp', 'Dm_total_save', 'GammaAmp', 'GammaSAmp', 'GammaS_total_save', 'Gamma_total_save', 'M', 'NmaxS', 'dG', 'dGS', 'frequencies', 'n_int', 'noiseAmpScaled', 'omegaS_total_save', 'omega_total_save'])

In [9]:
all_channels = pd.DataFrame(np.transpose(np.transpose(spectra['Dm_total_save'])[0]))
modes = [x[0]*np.shape(spectra['Dm_total_save'])[0] for x in spectra['omega_total_save']]

In [10]:
labeled_modes=[]
for i in range(3):
  print("ITERATION: "+str(i))
  amp=[]
  length = np.shape(spectra['Dm_total_save'])[0]
  modes = [x[i]*length for x in spectra['omega_total_save']]
  seeker = 0
  modes = np.sort(modes)
  for j in range(length):
    all_amp = [0]
    while seeker < len(modes) and modes[seeker] <= j+100:
        # print(np.exp(np.square((j-modes[seeker])/100)))
      all_amp.append(1/(np.exp(np.square((j-modes[seeker])/10))))
      seeker += 1
    while seeker < len(modes) and seeker >= 0 and modes[seeker] >= j-100:
      # print(modes[seeker])
      seeker -= 1
    seeker = max(seeker,0)
    amp.append(max(all_amp))
    if j %100000 ==0: print(j)
  for i in range(5):
    labeled_modes.extend(amp)
  print(len(labeled_modes))

ITERATION: 0
0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning:

overflow encountered in exp



100000
200000
300000
400000
500000
600000
700000
800000
900000
5000000
ITERATION: 1
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
10000000
ITERATION: 2
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
15000000


In [11]:
# training set generation
x = []
for i in range(3):
  all_channels = pd.DataFrame(np.transpose(np.transpose(spectra['Dm_total_save'])[i]))
  x.extend(np.array(all_channels[[0,1,2,3,4]]))
  x.extend(np.array(all_channels[[5,6,7,8,9]]))
  x.extend(np.array(all_channels[[10,11,12,13,14]]))
  x.extend(np.array(all_channels[[15,16,17,18,19]]))
  x.extend(np.array(all_channels[[20,21,22,23,24]]))

In [12]:
!pip freeze | grep plotly

plotly==4.14.3


In [15]:
pd.options.plotting.backend = "plotly" 

def show_spectra(view_range,spect,modes, spectra_sample=True):
  # take subset of plot
  t_modes = np.sort(modes)
  temp_modes = pd.Series(t_modes)[(pd.Series(t_modes) < view_range[1])]
  temp_modes = pd.Series(temp_modes)[(pd.Series(temp_modes) > view_range[0])]
  fig = []
  if spectra_sample:
    fig = pd.Series(spect).iloc[range(view_range[0],view_range[1])].plot(kind='line')
  else:
    fig = pd.Series(spect).plot(template='plotly_dark', kind='line')
    temp_modes -= view_range[0]
  for mode in temp_modes:
    fig.add_vline(x=mode, line_width=1, line_color="green") #requires plotly 4.12 and above
  fig.show()

ValueError: ignored

In [16]:
modes = [x[0]*length for x in spectra['omega_total_save']]
show_spectra((int(500000), int(501000)), labeled_modes, modes)

NameError: ignored

In [ ]:
def stop():
  spectra =  loadmat('Spectra.mat')#['Dm_total_save']

  labeled_modes=[]
  for i in range(1):
    print("ITERATION: "+str(i))
    amp=[]
    length = np.shape(spectra['Dm_total_save'])[0]
    modes = [x[i]*length for x in spectra['omega_total_save']]
    seeker = 0
    modes = np.sort(modes)
    for j in range(length):
      all_amp = [0]
      while seeker < len(modes) and modes[seeker] <= j+100:
          # print(np.exp(np.square((j-modes[seeker])/100)))
        all_amp.append(1/(np.exp(np.square((j-modes[seeker])/10))))
        seeker += 1
      while seeker < len(modes) and seeker >= 0 and modes[seeker] >= j-100:
        # print(modes[seeker])
        seeker -= 1
      seeker = max(seeker,0)
      amp.append(max(all_amp))
      if j %100000 ==0: print(j)
    labeled_modes.extend(amp)
    print(len(labeled_modes))

#y = labeled_modes
#x = spectra['Dm_total_save']
#window_len = 1000
#reshaped_y = np.reshape(y, (int(len(y)/window_len), window_len))
#reshaped_x = np.reshape(np.array(x), (int(len(x)/window_len), window_len, 5))

In [ ]:
y = labeled_modes
window_len = 1000
reshaped_y = np.reshape(y, (int(len(y)/window_len), window_len))
reshaped_x = np.reshape(np.array(x), (int(len(x)/window_len), window_len, 5))

In [ ]:
first_spectra = reshaped_x[0:1000]

In [ ]:
y_pred = new_model.predict(first_spectra)

In [ ]:
np.reshape(y_pred, (1, 1000000))

#modes = [x[0]*length for x in spectra['omega_total_save']]
show_spectra((int(500000), int(501000)), np.reshape(y_pred, (1, 1000000))[0], modes)

In [ ]:
show_spectra((int(500000), int(501000)), y, modes)

In [ ]:
#pd.Series(np.reshape(reshaped_y, (1, 15000000))[0][15000000 - 1000:]).plot()
#pd.Series(np.reshape(y_pred, (1, 15000000))[0][15000000 - 1000:]).plot()

In [ ]:
import seaborn as sns
from scipy.signal import savgol_filter
from scipy.signal import find_peaks

reshaped_y_pred = np.reshape(y_pred, (1, 1000000))[0]
#pd.Series(np.reshape(reshaped_y, (1, 1000000))[0][0:10000]).plot()
#pd.Series(savgol_filter(y_pred, 51, 3)[0:10000]).plot()
predicted_modes = find_peaks(reshaped_y_pred, height=0.65)[0]
#print(len(modes))
#sns.lineplot(data = pd.Series(np.reshape(reshaped_y, (1, 1000000))[0][0:10000]))

In [ ]:
print(len(predicted_modes))
predicted_modes

In [ ]:
modes = [x[0]*length for x in spectra['omega_total_save']]
len(modes)

# Bin Metrics

In [ ]:
import plotly.graph_objects as go

def bin_plot(num_bins):
  peaks_detected_count = []
  peaks_detected = []

  for s in range(num_bins):
   window_peaks = [i for i in predicted_modes if (i >= s * 1000000/num_bins) and (i < (s+1) * 1000000/num_bins)]
   peaks_detected_count.append(len(window_peaks))
   window_peaks = [i for i in modes if (i >= s * 1000000/num_bins) and (i < (s+1) * 1000000/num_bins)]
   peaks_detected.append(len(window_peaks))

  x_bins=list(range(num_bins))
  fig = go.Figure()
  # actual mode distribution by bin
  fig.add_trace(go.Scatter(x=x_bins, y=peaks_detected,
                    mode='lines+markers',
                    name='Actual values'))
  # predicted mode distribution by bin
  fig.add_trace(go.Scatter(x=x_bins, y=peaks_detected_count,
                    mode='lines+markers',
                    name='Predicted values'))
  fig.update_layout(title='Number of Modes by Bins of Normalized Hertz',
                  xaxis_title='Bin Number',
                   yaxis_title='Number of Modes')
  fig.show()

bin_plot(15)
bin_plot(30) 


In [ ]:
def bin_metrics(num_bins):
  peaks_detected_count = []
  peaks_detected = []
  for s in range(num_bins):
    window_peaks = [i for i in predicted_modes if (i >= s * 1000000/num_bins) and (i < (s+1) * 1000000/num_bins)]
    peaks_detected_count.append(len(window_peaks))
    window_peaks = [i for i in modes if (i >= s * 1000000/num_bins) and (i < (s+1) * 1000000/num_bins)]
    peaks_detected.append(len(window_peaks))

  diff = []
  total_missed = 0 
  pred_direction = 0
  for i in range(num_bins):
    diff.append(abs(peaks_detected_count[i] - peaks_detected[i]))
    total_missed += abs(peaks_detected_count[i] - peaks_detected[i])
    pred_direction += peaks_detected_count[i] - peaks_detected[i]

  avg_missed = sum(diff)/num_bins 
  avg_direction = int(pred_direction)/num_bins

  print("Each bin on average misses %f modes" % avg_missed)
  print("This model missed a total of %d modes" % total_missed)

  if pred_direction > 0:
    print("On average, each bin tends to overpredict the number of modes by %f" % avg_direction)
  else:
    print("On average, each bin tends to underpredict the number of modes by %f" % avg_direction)

bin_metrics(15)
bin_metrics(30)
